# Figure 2: Micromagnetics and Energy Landscape Evolution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from scipy.constants import physical_constants
import sys

sys.path.append('../..')
from Utils.PhaseDiagramFunctions import *
from Utils.QuantityReader import read_quantity

sys.path.append('../../Micromagnetics/SimulationsAndAnalysis')
from GeneralMicromagneticAnalysisTools import Plot


# Specify E0 and omega is to be used in this plot (in manuscript, E0=0.20, omega=1.18 were used)
E0 = 0.20
omega = 1.18


plt.style.use('Style.mplstyle')
paramsfile = '../../Micromagnetics/SimulationsAndAnalysis/MicromagneticParams'  # File in which micromagnetic simulation parameters are stored


def plot_energy_landscape(time_start_idx, ax, R_sol, eta_sol, Ez_array, Bz_array,
E_exchangeFitParams, E_magnetic_integralFitParams, E_electric_integralFitParams):
    
    """
    Plot the energy landscape on the axis ax supplied.
    """

    def get_total_energy(Ez, Bz):
        
        """
        Total energy i.e. Exchange + Zeeman + Electric energies.
        """

        exchange_energy = inverse_quadratic_linear_fit(R, E_exchangeFitParams[0], E_exchangeFitParams[1], E_exchangeFitParams[2], E_exchangeFitParams[3])
        magnetic_energy = -Bz * quadratic_fit(R, E_magnetic_integralFitParams[0], E_magnetic_integralFitParams[1], E_magnetic_integralFitParams[2])
        electric_energy = -Ez * np.cos(ETA) * linear_fit(R, E_electric_integralFitParams[0], E_electric_integralFitParams[1])
        
        return exchange_energy + magnetic_energy + electric_energy

    # Define grid of radii and helicities for the plotting of the energy surface
    R_3d = np.linspace(0.7, 2.7, 100)
    eta_3d = np.linspace(0., 2*np.pi, 100)
    R, ETA = np.meshgrid(R_3d, eta_3d)

    # Express the mesh in the cartesian system
    X, Y = R*np.cos(ETA), R*np.sin(ETA)

    # Minimum and maximum energy values to plot (adjust by hand on case-by-case basis)
    minE = 3.5
    maxE = 15

    R_idx = np.argmin(np.abs(R_sol[time_start_idx] - R_3d))
    eta_idx = np.argmin(np.abs(eta_sol[time_start_idx] % (2*np.pi) - eta_3d))
    energy_3d = get_total_energy(Ez_array[time_start_idx], Bz_array[time_idx])
    energy = energy_3d[eta_idx, R_idx]
    energy_3d[np.where(energy_3d > maxE)] = np.nan  # Clip the "horn"
    ax.plot_surface(X, Y, energy_3d, cmap=plt.cm.YlOrBr_r, alpha=0.5, vmin=minE, vmax=maxE)
    ax.scatter(R_sol[time_start_idx]*np.cos(eta_sol[time_idx]), R_sol[time_idx]*np.sin(eta_sol[time_idx]), energy, 'o', color='lime', s=100)

    ax.set_xlabel(r'$R\cos\eta$')
    ax.set_ylabel(r'$R\sin\eta$', labelpad=10)
    ax.zaxis.set_rotate_label(False)
    ax.set_zlabel(r'$\Delta U / 2\pi d$', labelpad=10, rotation=-90)
    ax.set_zlim(minE, maxE)


cc_directory = '../../CollectiveCoordinates/CollectiveCoordinateIntegration/CollectiveCoordinateDataSupplied'
micromagnetics_directory = '../../Micromagnetics/SimulationsAndAnalysis/MicromagneticsDataSupplied'

R_sol = np.load(cc_directory + '/R/E0%.2fomega%.2f.npy' %(E0, omega))
eta_sol = np.load(cc_directory + '/eta/E0%.2fomega%.2f.npy' %(E0, omega))


fig = plt.figure(figsize=(20, 10))
gs = GridSpec(2, 4, figure=fig)

# Obtain parameters such as magnetic field, fit parameters (see Methods section of manuscript on fitting)
# E_exchange, E_magnetic, and E_electric are the contributions from exchange, Zeeman, and electric field to energy respectively
# alpha is the Gilbert damping constant
# F_rex is -dU_ex / dR
# Gamma11 and Gamma22 are as in the manuscript where 1 is R and 2 is η
# G12 is the G_{Rη} from the manuscript

# What I call Xi are parameters that show up in the analytical expressions for the helicity-dependent
# parts of the generalised forces for the skyrmion profile ansatz we use

# Xi1 is the integral of cos(2θ) dθ / dR over ρ
# Xi2 is the integral of ρ * d^2θ / dRdρ over ρ
# Xi3 is the integral of cos(θ)sin(θ) over ρ
# Xi4 is the integral of ρ * dθ / dρ over ρ
Bz, dw_width, E_exchangeFitParams, E_magnetic_integralFitParams, E_electric_integralFitParams, alpha, F_RexFitParams, Gamma11FitParams, Gamma22FitParams, G12FitParams, Xi1FitParams, Xi2FitParams, Xi3FitParams, Xi4FitParams, optimalSkyrmionRadius = read_parameters(cc_directory)

times = np.load(cc_directory + '/General/times.npy')

# Array of magnetic and electric fields
Bz_array = np.full_like(times, Bz)
Ez_array = E0 * np.cos(omega*times)

# The index of the time we start at is where the helicity is closest to zero, but the time is greater than half the total integration time
start_idx = np.argmin(np.abs(eta_sol[np.where(times > times[len(times)//2])] % (2*np.pi)))
time_start = times[start_idx] + times[len(times)//2]

# For labelling with times as a fraction of the full period
time_texts = {0: r'$t = t_0$', 1: r'$t = t_0 + T/4$', 2: r'$t = t_0 + T/2$', 3: r'$t = t_0 + 3T / 4$'}

# Loop through the four time points displayed
for i in range(4):
    
    # Calculate the time (in dimensionless units)
    period = 2*np.pi / omega
    time = time_start + i * period / 4

    # Convert time to index in simulation
    time_idx = np.argmin(np.abs(times - time))

    # Plot the energy landscape
    energy_ax = fig.add_subplot(gs[1, i], projection='3d')
    plot_energy_landscape(time_idx, energy_ax, R_sol, eta_sol, Ez_array, Bz_array, 
                        E_exchangeFitParams, E_magnetic_integralFitParams, E_electric_integralFitParams)
    
    # Rest of code is plotting the micromagnetics in bottom row
    micromagnetics_ax = fig.add_subplot(gs[0, i])
    micromagnetics_simulation_directory = micromagnetics_directory + '/E0%.2fomega%.2f' %(E0, omega)

    # Conversions from SI to rescaled/dimensionless units
    Ms = read_quantity('Ms', paramsfile)
    I1 = read_quantity('I1', paramsfile)
    I2 = read_quantity('I2', paramsfile)
    gamma = physical_constants[u'electron gyromag. ratio'][0]
    time_multiple = Ms * I2 / (gamma*I1*I1)
    field_multiple = I1**2 / (Ms * I2)
    length_multiple = np.sqrt(I2 / I1)

    # Use MicromagneticAnalysisTools.Plot for plotting
    plotter = Plot.MagnetizationPlotter('magnetization', micromagnetics_simulation_directory + '/Data', 'm%06d.ovf' %time_idx, ax=micromagnetics_ax, length_units=length_multiple, interpolation='bicubic', 
                                            plot_quiver=True, step=2, quiver_colour=[1, 1, 1], quiver_scale=3, quiver_headwidth=10, quiver_headlength=20, quiver_headaxislength=20)

    colour_plot, quiver_plot = plotter.plot()

    # Rescale axis lengths
    plot_extent = colour_plot.get_extent()
    centre_X = (plot_extent[1] - plot_extent[0]) / 2
    centre_Y = (plot_extent[3] - plot_extent[2]) / 2
    colour_plot.set_extent([plot_extent[0] - centre_X, plot_extent[1] - centre_X, plot_extent[2] - centre_Y, plot_extent[3] - centre_Y])
    quiver_plot.XY[:, 0] = quiver_plot.XY[:, 0] - centre_X
    quiver_plot.XY[:, 1] = quiver_plot.XY[:, 1] - centre_Y

    axis_ticks = [i for i in range(-4, 5, 2)]

    micromagnetics_ax.set_xticks(axis_ticks)
    micromagnetics_ax.set_yticks(axis_ticks)

    micromagnetics_ax.set_xlim(np.min(axis_ticks), np.max(axis_ticks))
    micromagnetics_ax.set_ylim(np.min(axis_ticks), np.max(axis_ticks))

    micromagnetics_ax.set_xlabel(r'$x$')
    micromagnetics_ax.set_ylabel(r'$y$')

    micromagnetics_ax.set_title(time_texts[i], size=28, ha='center', pad=20)

plt.tight_layout(pad=5)
plt.show()